In [236]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [238]:
df = pd.DataFrame()
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head(3)

,review,sentiment
0,I think the problem with this show not getting...,1
1,"Having finally caught up with this ""masterpiec...",1
2,"Watching this odd little adventure movie, it's...",1


In [0]:
X_train = df.loc[:24999, 'review'].values
y_train = df.loc[:24999, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer_obj = Tokenizer()
total_reviews = X_train + X_test
tokenizer_obj.fit_on_texts(total_reviews)

In [0]:
# pad sequences
max_length = max([len(s.split()) for s in total_reviews])

In [243]:
print(max_length)

2795


In [0]:
# define vocabulary size
vocab_size = len(tokenizer_obj.word_index) + 1

In [0]:
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test)

In [0]:
X_train_pad = pad_sequences(X_train_tokens, maxlen = max_length, padding = 'post')
X_test_pad = pad_sequences(X_test_tokens, maxlen = max_length, padding = 'post')

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

In [0]:
EMBEDDING_DIM = 100

In [0]:
# model = Sequential()
# model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length = max_length))
# model.add(GRU(units = 32, dropout = 0.2, recurrent_dropout = 0.2))
# model.add(Dense(1, activation = 'sigmoid'))

In [0]:
# model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# model.summary()

In [0]:
# model.fit(X_train_pad, y_train, batch_size = 128, epochs = 25, validation_data = (X_test_pad, y_test), verbose = 1)

In [0]:
# # Test Samples
# test_sample_1 = "This movie is fantastic! I really like it becuse it is so good!"
# test_sample_2 = "Good movie!"
# test_sample_3 = "Maybe I like this movie."
# test_sample_4 = "Not to my taste, will skip and watch another movie"
# test_sample_5 = "if you like action, then this moview might be good for you."
# test_sample_6 = "Bad movie!"
# test_sample_7 = "Not a good movie!"
# test_sample_8 = "This movie really sucks! Can I get my money back please?"
# test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

In [0]:
# test_samples_tokens = tokenizer_obj.texts_to_sequences(test_samples)
# test_samples_tokens_pad = pad_sequences(test_samples_tokens, maxlen = max_length)

In [0]:
# # 1 - positive sentiment, 0 - negative sentiment
# model.predict(x=test_samples_tokens_pad)

In [1]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
review_lines = list()
lines = df['review'].values.tolist()

In [0]:
stop_words = stopwords.words('english')

In [257]:
i = 0
for line in lines:
  tokens = word_tokenize(line)
  
  if (i % 500 == 0):
    print(i)
  
  # convert to lower case
  tokens = [w.lower() for w in tokens]
  
  #remove punctuation from each word
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  
  # filter out stop words
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  review_lines.append(words)
  
  i+= 1

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500


In [258]:
len(review_lines)

50000

In [0]:
import gensim

In [260]:
# train word2vec model
model = gensim.models.Word2Vec(sentences=review_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)

# vocab size
words = list(model.wv.vocab)
print("Vocab size: ", len(words))

Vocab size:  134156


In [261]:
model.wv.most_similar('horrible')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.9134328961372375),
 ('awful', 0.8634687066078186),
 ('horrendous', 0.7730600237846375),
 ('pathetic', 0.7684623003005981),
 ('atrocious', 0.7541861534118652),
 ('dreadful', 0.7468248605728149),
 ('horrid', 0.7428416013717651),
 ('sucks', 0.7411549091339111),
 ('lousy', 0.73479163646698),
 ('bad', 0.7198077440261841)]

In [262]:
model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])

[('princess', 0.8736845850944519),
 ('prince', 0.8554849028587341),
 ('romeo', 0.854260265827179),
 ('juliet', 0.8492663502693176),
 ('tromeo', 0.8481894135475159),
 ('queen', 0.8371779918670654),
 ('burnford', 0.8369131088256836),
 ('lennie', 0.8356282114982605),
 ('damnedthe', 0.8349870443344116),
 ('chatterley', 0.8345091938972473)]

In [263]:
model.wv.doesnt_match("woman queen king movie".split())

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'movie'

In [264]:
filename = 'imdb_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import os

In [0]:
embeddings_index = {}
f = open(os.path.join('', 'imdb_embedding_word2vec.txt'), encoding='utf-8')

In [267]:
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddings_index[word] = coefs
f.close

<function TextIOWrapper.close>

In [0]:
# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(review_lines)
sequences = tokenizer_obj.texts_to_sequences(review_lines)

In [269]:
# pad sequences
word_index = tokenizer_obj.word_index
print("Found %s unique tokens.", len(word_index))

Found %s unique tokens. 134156


In [270]:
review_pad = pad_sequences(sequences, maxlen=max_length)
sentiment = df['sentiment'].values

print('Shape of review tensor: ', review_pad.shape)
print("Shape of sentiment tensor: ", sentiment.shape)

Shape of review tensor:  (50000, 2795)
Shape of sentiment tensor:  (50000,)


In [0]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

In [0]:
for word, i in word_index.items():
  if i > num_words:
    continue
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [273]:
num_words

134157

In [0]:
from keras.initializers import Constant

In [0]:
model = Sequential()
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    embeddings_initializer = Constant(embedding_matrix),
    input_length = max_length,
    trainable = False
)
model.add(embedding_layer)
model.add(GRU(units = 32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = 'sigmoid'))

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 2795, 100)         13415700  
_________________________________________________________________
gru_4 (GRU)                  (None, 32)                12768     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 13,428,501
Trainable params: 12,801
Non-trainable params: 13,415,700
_________________________________________________________________


In [0]:
VALIDATION_SPLIT = 0.2

In [0]:
indices = np.arange(review_pad.shape[0])
np.random.shuffle(indices)
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

In [0]:
X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [216]:
print("Shape of X_train_pad_tensor ", X_train_pad.shape)

Shape of X_train_pad_tensor  (40000, 2795)


In [217]:
y_train.shape

(40000,)

In [218]:
X_test_pad.shape

(10000, 2795)

In [219]:
y_test.shape

(10000,)

In [220]:
model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose = 1)

AttributeError: ignored

In [221]:
# accuaracy on 6 epochs around 0.8651
model.save('trained_on_6_epochs.h5')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# accuracy on 10 epochs around 0.8766
model.save('trained_on_10_epochs.h5')

In [0]:
from keras.models import load_model

In [0]:
model = load_model('trained_on_10_epochs.h5')

In [0]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer_obj = pickle.load(handle)

In [0]:
predict_lines = list()
lines = ["This was a awesome! Like it! Watch it ASAP!", "Hmm, this was pretty bad. Didn't like it fellas", "It's okay, but nothing special"]

In [0]:
i = 0
for line in lines:
  tokens = word_tokenize(line)
  
  # convert to lower case
  tokens = [w.lower() for w in tokens]
  
  #remove punctuation from each word
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  
  # remove remaining tokens that are not alphabetic
  words = [word for word in stripped if word.isalpha()]
  
  # filter out stop words
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  predict_lines.append(words)
  

In [0]:
predict_sequences = tokenizer_obj.texts_to_sequences(predict_lines)
predict_pad = pad_sequences(predict_sequences, maxlen=max_length)

In [292]:
model.predict(predict_pad)

array([[0.89841706],
       [0.16635819],
       [0.20810091]], dtype=float32)

In [283]:
tokenizer_obj

In [0]:
import pickle

In [0]:
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer_obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
del model

In [0]:
del tokenizer_obj

In [0]:
stop_words

In [0]:
# saving
with open('stop_words.pickle', 'wb') as handle:
    pickle.dump(stop_words, handle, protocol=pickle.HIGHEST_PROTOCOL)